# NLSub

In [ ]:
import json

import numpy as np
import scipy.stats as sstats
import tensorflow as tf
import matplotlib.pyplot as plt

from gwpy.timeseries import TimeSeries
from sklearn.model_selection import train_test_split

## Load data

In [ ]:
fs = 512  # sampling rate

def load_data(channels):
    
    dset = []
    norm_factors = []
    
    for channel in channels:
        
        # load data
        fname = f'data/{channel}' 
        data = np.load(fname)
        
        # remove sides due to whitening artifacts
        gps_times = data[fs*4:-fs*4, 0]
        data_tseries = data[fs*4:-fs*4, 1].reshape(-1, 1)
        
        # normalize and append tseries
        norm_factor = np.max(np.abs(data_tseries))
        norm_factors.append(norm_factor)
        dset.append(data_tseries / norm_factor)
    
    dset = np.squeeze(dset)
    dset = np.float32(dset)
    gps_times = np.array(gps_times)
    
    return dset, gps_times, norm_factors


# load training data
channels = ['DCS-CALIB_STRAIN_CLEAN_C01_512Hz_27hrs_whitened.npy',
            'LSC-POP_A_RF45_I_ERR_DQ_512Hz_27hrs_whitened.npy',
            'LSC-POP_A_RF45_Q_ERR_DQ_512Hz_27hrs_whitened.npy',
            'LSC-POP_A_RF9_I_ERR_DQ_512Hz_27hrs_whitened.npy']

dset, gps_times, norm_factors = load_data(channels)


# load event data
channels_event = ['DCS-CALIB_STRAIN_CLEAN_C01_512Hz_event_120s_whitened.npy',
            'LSC-POP_A_RF45_I_ERR_DQ_512Hz_event_120s_whitened.npy',
            'LSC-POP_A_RF45_Q_ERR_DQ_512Hz_event_120s_whitened.npy',
            'LSC-POP_A_RF9_I_ERR_DQ_512Hz_event_120s_whitened.npy']

dset_event, gps_times_event, norm_factors_event = load_data(channels_event)

## Data preparation

### Select noisy times
Algorithm fails to learn when all 27hrs of data is given because it's mostly Gaussian. Thus here we find times when an aux channel is non-Gaussian, and use these noisy times to get intput/output arrays.

In [ ]:
z_threshold = 11.7  # threshold factor found by trial
noise_idx = np.argwhere(np.abs(sstats.zscore(dset[1])) > z_threshold).T.tolist()[0]

clear_size = fs * 4  # amount of data around to be considered 'related' around a data point

# taken from https://stackoverflow.com/questions/53177358/removing-numbers-which-are-close-to-each-other-in-a-list
usedValues = set()
noise_idx_isolated = []

for v in noise_idx:
    if v not in usedValues:
        noise_idx_isolated.append(v)

        for lv in range(v - clear_size, v + clear_size+1):
            usedValues.add(lv)

print(f'{len(noise_idx_isolated)} noisy places')

### Create input/output arrays

In [ ]:
# input filter params for the DNN
rec_size = 768
rec_future = 256
rec_size = 48
rec_future = 16

def get_arrays(dset, input_values, output_values, box_start, box_end, rec_size, rec_future):
    rec_past = rec_size - rec_future
    for i in range(box_start+rec_past, box_end-rec_future):
        array = np.array([dset[1, i-rec_past:i+rec_future],
                          dset[2, i-rec_past:i+rec_future],
                          dset[3, i-rec_past:i+rec_future]])
        #array = np.array([dset[1, i-rec_past:i+rec_future]])
        input_values.append(array)
    output_values.append(dset[0, box_start+rec_past:box_end-rec_future])
    
    return input_values, output_values

input_values = []
output_values = []

for idx in noise_idx_isolated:
    box_start = idx - clear_size
    box_end = idx + clear_size
    input_values, output_values = get_arrays(dset, input_values, output_values, box_start,
                                             box_end, rec_size, rec_future)
    
output_values = np.hstack(output_values).reshape(-1,1)
input_values = np.array(input_values)

In [ ]:
output_values.shape, input_values.shape

### Remove glitches
Need also to remove times when the strain data contains huge glitches which are not caused by our aux channels. 

In [ ]:
glitch_threshold = 40  # threshold factor found by trial
glitches_idx = np.argwhere(np.abs(sstats.zscore(output_values)) > glitch_threshold).T.tolist()[0]

# taken from https://stackoverflow.com/questions/53177358/removing-numbers-which-are-close-to-each-other-in-a-list
usedValues = set()
glitches_idx_isolated = []

for v in glitches_idx:
    if v not in usedValues:
        glitches_idx_isolated.append(v)

        for lv in range(v - clear_size, v + clear_size+1):
            usedValues.add(lv)

for idx in glitches_idx_isolated[::-1]:  # use REVERSED list
    
    idx_start = idx - clear_size - rec_size - rec_future
    idx_end = idx + clear_size + rec_future
        
    output_values = np.delete(output_values, slice(idx_start, idx_end), axis = 0)
    input_values = np.delete(input_values, slice(idx_start, idx_end), axis = 0)
    
print(f'Removed {len(glitches_idx_isolated)} glitches')

### Normalization
After selecting noisy aux parts and removing times where strain data contains huge glitches, we need to re-normalize the data.

Input values (i.e. aux channel data) don't need normalization because they contain the noisiest parts, so np.max(np.abs(..)) is already 1. Also note that the event data needs to be normalized with respect to the training data set, so that these two data sets would be consistent with each other.

In [ ]:
norm_factor_output = np.max(np.abs(output_values))
output_values = output_values / norm_factor_output
# note that input values dont need to be normalised since all of them have max val of 1

# normalize event data
dset_event_norm = []
dset_event_norm.append(dset_event[0] * norm_factors_event[0]/norm_factors[0]/norm_factor_output)
dset_event_norm.append(dset_event[1] * norm_factors_event[1]/norm_factors[1])
dset_event_norm.append(dset_event[2] * norm_factors_event[2]/norm_factors[2])
dset_event_norm.append(dset_event[3] * norm_factors_event[3]/norm_factors[3])
dset_event_norm = np.array(dset_event_norm)

# normalisation to be used to keep event and 27hrs datasets consistent; also used when coloring the data
normalisation_values = [norm_factors[0]*norm_factor_output, norm_factors[1], norm_factors[2], norm_factors[3]]
print(normalisation_values)

Create input/output arrays for the event

In [ ]:
input_values_event = []
output_values_event = []
input_values_event, output_values_event = get_arrays(dset_event_norm, input_values_event, 
                                                     output_values_event, 0, 
                                                     len(dset_event_norm[0,:]), rec_size, 
                                                     rec_future)

input_values_event = np.array(input_values_event)
output_values_event = np.hstack(output_values_event).reshape(-1,1)
output_gps_times_event = gps_times_event[rec_size-rec_future:-rec_future]

### Get training, validation and testing arrays

In [ ]:
seed = 0
train_input, test_input = train_test_split(input_values, test_size=0.1, shuffle=True, random_state=seed)
train_output, test_output = train_test_split(output_values, test_size=0.1, shuffle=True, random_state=seed)
test_input = input_values_event
test_output = output_values_event

print('Training dset input/output shapes: ', train_input.shape, train_output.shape)
print('Validating dset input/output shapes: ', valid_input.shape, valid_output.shape)
print('Testing dset input/output shapes: ', test_input.shape, test_output.shape)

## Creating and training a model

In [ ]:
width = 1024
l2 = 1e-4
batch = 10000
epochs = 500
lrate = 1e-3

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  lrate,
  decay_steps=80,
  decay_rate=10,
  staircase=True)

# MODEL
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(width, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(width, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)),
    tf.keras.layers.Dense(1),
])

# COMPILE AND FIT
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule))
    
model_fit = model.fit(train_input, train_output, validation_data=(valid_input, valid_output), batch_size = batch, epochs = epochs, verbose = 2)
    
# MAKE PREDICTION
prediction = model.predict(test_input)

## Plots

In [ ]:
output_suffix = 'model'

# PLOTTING
gps = 1264316116 + 40
plot_win = 50
start_plot = gps - plot_win
end_plot = gps + plot_win
tseries_orig = TimeSeries(np.squeeze(output_values_event), times=output_gps_times_event)
tseries_orig = tseries_orig.crop(start_plot,end_plot)
tseries_pred = TimeSeries(np.squeeze(prediction), times=output_gps_times_event)
tseries_pred = tseries_pred.crop(start_plot,end_plot)
tseries_clean = TimeSeries(np.squeeze(output_values_event) - np.squeeze(prediction), times=output_gps_times_event)
tseries_clean = tseries_clean.crop(start_plot,end_plot)

plt.figure(figsize=(10,10))
plt.subplot(3,1,1)
plt.plot(tseries_pred, label='predicted')
plt.legend()
plt.subplot(3,1,2)
plt.plot(tseries_orig, label='real')
plt.plot(tseries_pred, label='predicted')
plt.legend()
plt.subplot(3,1,3)
plt.plot(tseries_clean, label='real - predicted')
plt.legend()
plt.savefig(f'output/plots/training_{output_suffix}_tseries.png')

# -------------------------------------
def make_oscan(gps, oscan_name):
    plot_win = 5
    start_plot = gps - plot_win
    end_plot = gps + plot_win

    dataset = ['orig','clean','diff']
    q_trans = {}
    q_trans['orig'] = tseries_orig.q_transform(outseg=(start_plot,end_plot),qrange=(10,20))
    q_trans['clean'] = tseries_clean.q_transform(outseg=(start_plot,end_plot),qrange=(10,20))
    q_trans['diff'] = q_trans['orig'] - q_trans['clean']

    ylim = (20, 200) 
    alim = (0, 25)

    label = {}
    label['orig'] = 'Original data'
    label['clean'] = 'Cleaned data'
    label['diff'] = 'Original - Cleaned'

    plot, axes = plt.subplots(nrows=3, sharex=True, figsize=(3.375*2.0,3.375*3.0))

    for i, ax in zip(dataset,axes):

        pcm = ax.imshow(q_trans[i],vmin=alim[0],vmax=alim[1])
        ax.set_ylim(ylim[0],ylim[1])
        ax.set_xlabel('')
        ax.set_yscale('log')
        ax.plot([gps],10, label=label[i], visible=False)
        ax.grid(alpha=0.6)
        ax.legend(loc='upper left', handlelength=0, handletextpad=0)

    axes[1].set_ylabel(r"$\mathrm{Frequency \ (Hz)}$")
    cbar = axes[0].colorbar(clim=(alim[0], alim[1]),location='top')
    cbar.set_label(r"$\mathrm{Normalized \ energy}$")
    plt.savefig(f'output/plots/{oscan_name}.png')

gps = 1264316122
oscan_name = f'training_{output_suffix}_oscan_1'
make_oscan(gps, oscan_name)
gps = 1264316154
oscan_name = f'training_{output_suffix}_oscan_2'
make_oscan(gps, oscan_name)
gps = 1264316164
oscan_name = f'training_{output_suffix}_oscan_3'
make_oscan(gps, oscan_name)

# -------------------------------------
plt.figure()
plt.semilogy(model_fit.history['loss'], label='train')
plt.semilogy(model_fit.history['val_loss'], label='valid')
plt.savefig(f'output/plots/training_{output_suffix}_hist.png')

# Save outputs

In [ ]:
model.save(f'output/{output_suffix}')

# save various params, including the normalisation values
json_dict = {'norm_values': normalisation_values, 'fs': fs, 'rec_size':rec_size, 'rec_future':rec_future}
with open(f'output/{output_suffix}/dataset_params.json', 'w') as outfile:                                    
    json.dump(json_dict, outfile)
